In [1]:
%pwd

'd:\\data_science\\MLOPS\\Red-Wine-Quality\\research'

In [2]:
import os
os.chdir("../")
%pwd

'd:\\data_science\\MLOPS\\Red-Wine-Quality'

In [3]:
import pandas as pd
from pathlib import Path
from dataclasses import dataclass

In [4]:
data=pd.read_csv("artifacts/data_transformation/train.csv")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,8.4,0.310,0.29,3.1,0.194,14.0,26.0,0.99536,3.22,0.78,12.0,6
1,8.0,0.390,0.30,1.9,0.074,32.0,84.0,0.99717,3.39,0.61,9.0,5
2,8.4,0.520,0.22,2.7,0.084,4.0,18.0,0.99682,3.26,0.57,9.9,6
3,8.0,0.745,0.56,2.0,0.118,30.0,134.0,0.99680,3.24,0.66,9.4,5
4,9.3,0.655,0.26,2.0,0.096,5.0,35.0,0.99738,3.25,0.42,9.6,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1194,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
1195,9.0,0.470,0.31,2.7,0.084,24.0,125.0,0.99840,3.31,0.61,9.4,5
1196,8.6,0.635,0.68,1.8,0.403,19.0,56.0,0.99632,3.02,1.15,9.3,5
1197,12.4,0.400,0.51,2.0,0.059,6.0,24.0,0.99940,3.04,0.60,9.3,6


In [5]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    depth: int
    iterations: int 
    learning_rate: float
    model_name: str
    target_column: str


In [6]:
os.chdir("d:\\data_science\\MLOPS\\Red-Wine-Quality\\src")
%pwd

'd:\\data_science\\MLOPS\\Red-Wine-Quality\\src'

In [ ]:
from src.utils.common import read_yaml, create_directories
from src.constants import *

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params=self.params.CatBoostRegressor
        schema=self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name=config.model_name,
            depth=params.depth,
            iterations=params.iterations,
            learning_rate=params.learning_rate,
            target_column=schema.name
        )

        return model_trainer_config

In [ ]:
import os
from src import logger
from catboost import CatBoostRegressor
import joblib

In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        X_train=train_data.drop([self.config.target_column],axis=1)
        X_test=test_data.drop([self.config.target_column], axis=1)
        y_train=train_data[self.config.target_column]
        y_test=test_data[self.config.target_column]


        catr = CatBoostRegressor(iterations=self.config.iterations, depth=self.config.depth, learning_rate=self.config.learning_rate, random_state=42)
        catr.fit(X_train,y_train)

        joblib.dump(catr,os.path.join(self.config.root_dir,self.config.model_name))

In [11]:
os.chdir("../")
%pwd

'd:\\data_science\\MLOPS\\Red-Wine-Quality'

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-03-26 22:30:43,768: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-26 22:30:43,772: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-26 22:30:43,777: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-26 22:30:43,780: INFO: common: created directory at: artifacts]
[2025-03-26 22:30:43,783: INFO: common: created directory at: artifacts/model_trainer]
0:	learn: 0.7893171	total: 170ms	remaining: 28.8s
1:	learn: 0.7753653	total: 181ms	remaining: 15.3s
2:	learn: 0.7622376	total: 191ms	remaining: 10.7s
3:	learn: 0.7503576	total: 193ms	remaining: 8.06s
4:	learn: 0.7389610	total: 204ms	remaining: 6.76s
5:	learn: 0.7282178	total: 214ms	remaining: 5.88s
6:	learn: 0.7187602	total: 224ms	remaining: 5.25s
7:	learn: 0.7081878	total: 234ms	remaining: 4.76s
8:	learn: 0.6998511	total: 243ms	remaining: 4.38s
9:	learn: 0.6905404	total: 254ms	remaining: 4.08s
10:	learn: 0.6820640	total: 264ms	remaining: 3.84s
11:	learn: 0.6742992	total: 27